In [48]:
import gym
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from gym.envs.registration import register
# 아래의 방식을 통해 게임을 등록함. 특히 is_slippery는 매우 중요함. 
"""
- 새로운 게임 형식을 만들어줌.
"""
register(
    id='FrozenLakeNotSlippery-v1',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    #max_episode_steps=100,
    #reward_threshold=0.78, # optimum = .8196
)

In [80]:
"""

- 사실 이런 형태의 미로에서는 reward를 끝나봐야 얻습니다. 끝날때까지 reward가 어디있는지 모른다는 이야기죠. 
- 비슷하게, 장기 바둑도 마찬가지입니다. 이전의 어떤 수가 reward를 결정했는지는 알기가 어려우니까요.
- 아무튼, 그러므로, dummy Q-learning algorithm에서는 t+1 state의 reward의 max값을 t state의 reward의 max 값으로 정합니다. 

"""
def rargmax(vector):
    m = np.amax(vector)
    #m = vector.max()
    indices = np.nonzero(vector==m)[0]
    return np.random.choice(indices)
# how to is slippely false
env = gym.make("FrozenLakeNotSlippery-v1")
observation = env.reset()

Q = np.zeros([env.observation_space.n, env.action_space.n])
num_episodes = 10000 # 일종의 epoch, 혹은 라이프.

rlist = []
for i in range(0, num_episodes):
    state = env.reset()
    #env.render()
    rAll = 0
    done = False
    while not done:
        action = rargmax(Q[state, :])
        new_state, reward, done, _ = env.step(action)
        Q[state, action] = reward + np.max(Q[new_state, :])

        rAll+=reward
        state = new_state 
    rlist.append(rAll)
Q

array([[ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.]])

1.0